In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import env

# DBSCAN import
from sklearn.cluster import DBSCAN
# Scaler import
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Import .txt file and convert it to a DataFrame object
df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None,
                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [4]:
df.head()

,date,time,page,id,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [11]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
        return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [16]:
df2 = pd.read_sql('''            
                            SELECT id as cohort, start_date, end_date, program_id 
                            FROM curriculum_logs.cohorts;  ''', get_connection('curriculum_logs'))

In [18]:
logs = pd.merge(df, df2, how='left', left_on='cohort', right_on='cohort')

In [47]:
logs.date = pd.to_datetime(logs.date, logs.start_date, logs.end_date)

In [28]:
logs.start_date = pd.to_datetime(logs.start_date)

In [29]:
logs.end_date = pd.to_datetime(logs.end_date)

In [43]:
logs.head()

,date,time,page,id,cohort,ip,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-08,2018-05-17,2.0


In [42]:
(logs['date'] >= logs.start_date) & (logs['date'] <= logs.end_date)


0         False
1         False
2         False
3         False
4          True
          ...  
900218    False
900219    False
900220    False
900221    False
900222    False
Length: 900223, dtype: bool

In [44]:
logs['active'] = np.where((logs['date'] >= logs.start_date) & (logs['date'] <= logs.end_date), 1, 0)

In [46]:
logs.active.value_counts()

1    647496
0    252727
Name: active, dtype: int64

In [1]:
# test function to get it all!
import wrangle


In [2]:
df = wrangle.wrangle_logs()

/Users/danchurchill/codeup-data-science/anomaly_detection_project/wrangle.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None,


In [3]:
df

,date,time,page,id,cohort,ip,start_date,end_date,program_id,active
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0,0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0,0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0,0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0,0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-08,2018-05-17,2.0,1
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0,0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0,0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0,0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,2014-02-04,2014-02-04,2.0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   time        900223 non-null  object        
 2   page        900222 non-null  object        
 3   id          900223 non-null  int64         
 4   cohort      847330 non-null  float64       
 5   ip          900223 non-null  object        
 6   start_date  847330 non-null  datetime64[ns]
 7   end_date    847330 non-null  datetime64[ns]
 8   program_id  847330 non-null  float64       
 9   active      900223 non-null  int64         
dtypes: datetime64[ns](3), float64(2), int64(2), object(3)
memory usage: 75.5+ MB
